In [1]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import random

In [2]:
# Fix random seed
seed = 42
np.random.seed(seed)
random.seed(seed)

# Pre-processing input data

In [3]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [4]:
results_df = pd.read_csv('../results/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [5]:
# Make the target dataset
target_df = results_df[['total_time', 'brand_raw', 'count', 'l2_cache_size', 'l3_cache_size', 'l2_cache_line_size', 'l2_cache_associativity', 'ghz_advertised_friendly', 'benchmark']].copy()
# Rename columns to *_target
target_df = target_df.rename(columns={
    'total_time': 'total_time_target',
    'brand_raw': 'brand_raw_target',
    'count': 'count_target',
    'l2_cache_size': 'l2_cache_size_target',
    'l3_cache_size': 'l3_cache_size_target',
    'l2_cache_line_size': 'l2_cache_line_size_target',
    'l2_cache_associativity': 'l2_cache_associativity_target',
    'ghz_advertised_friendly': 'ghz_advertised_friendly_target',
})

dataset_df = pd.merge(results_df, target_df, how='inner', on='benchmark')
dataset_df = dataset_df[dataset_df['brand_raw'] != dataset_df['brand_raw_target']]
dataset_df.head(2)

,total_time,total_cpu_usage,max_ram_usage,brand_raw,count,l2_cache_size,l3_cache_size,l2_cache_line_size,l2_cache_associativity,benchmark,ghz_actual_friendly,ghz_advertised_friendly,total_time_target,brand_raw_target,count_target,l2_cache_size_target,l3_cache_size_target,l2_cache_line_size_target,l2_cache_associativity_target,ghz_advertised_friendly_target
5,13.49,0.99,1435.851562,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.0996,2.9,32.01,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496
6,13.49,0.99,1435.851562,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.0996,2.9,19.09,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496


In [6]:
# remove one computer for testing
g_train = dataset_df[(dataset_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (dataset_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['benchmark','brand_raw', 'brand_raw_target'])
g_test = dataset_df[dataset_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['benchmark','brand_raw', 'brand_raw_target'])

In [7]:
mm_df = dataset_df[dataset_df['benchmark']=='MATRIX_MULT'].drop(columns=['benchmark'])
# remove one computer for testing
mm_train = mm_df[(mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (mm_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['brand_raw', 'brand_raw_target'])
mm_test = mm_df[mm_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw', 'brand_raw_target'])

In [8]:
st_df = dataset_df[dataset_df['benchmark']!='MATRIX_MULT'].drop(columns=['benchmark','count'])
# remove one computer for testing
st_train = st_df[(st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (st_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['brand_raw', 'brand_raw_target'])
st_test = st_df[st_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw', 'brand_raw_target'])

In [9]:
target = 'total_time_target'
features = mm_test.columns.copy().drop(target)
features_st = features.copy().drop(['count', 'count_target'])

In [10]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
scaler = StandardScaler()
X_g_train = scaler.fit_transform(X_g_train)
X_g_test = scaler.transform(X_g_test)

In [11]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
scaler = StandardScaler()
X_st_train = scaler.fit_transform(X_st_train)
X_st_test = scaler.transform(X_st_test)

In [12]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
scaler = StandardScaler()
X_mm_train = scaler.fit_transform(X_mm_train)
X_mm_test = scaler.transform(X_mm_test)

# Model

In [13]:
def objective(trial: optuna.Trial, X_train, y_train, X_test, y_test):
	dtrain = xgb.DMatrix(X_train, label=y_train)
	dtest = xgb.DMatrix(X_test, label=y_test)
	
	# Definimos los hiperparámetros a buscar (XGBoost)
	param = {
		"verbosity": 0,
		"device": "gpu",
		"objective": "reg:squarederror",
		"eval_metric": "rmse",
		"booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
		# L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
		"learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
	}
    
	if param["booster"] in ["gbtree", "dart"]:
		# maximum depth of the tree, signifies complexity of the tree.
		param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
		# minimum child weight, larger the term more conservative the tree.
		param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
		param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
		# defines how selective algorithm is.
		param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
		param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

	if param["booster"] == "dart":
		param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
		param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
		param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
		param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

	# training
	model = xgb.train(param, dtrain)
	# evaluation
	preds = model.predict(dtest)
	mse = mean_squared_error(y_test, preds)
	rmse = np.sqrt(mse)
	print(f"Trial: {trial.number} - MSE: {mse} - RMSE: {rmse}")
	return mse

# Hyperparameters Optimization

In [14]:
n_trials = 100

## General

In [15]:
# configuration optuna
study_g = optuna.create_study(direction='minimize')
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test), n_trials=n_trials)

[I 2024-06-20 04:38:27,487] A new study created in memory with name: no-name-ee57f021-0397-4a19-8aaf-944210ef2548
[I 2024-06-20 04:38:27,669] Trial 0 finished with value: 206.55344990167336 and parameters: {'booster': 'gbtree', 'lambda': 1.0683649546325783e-07, 'alpha': 0.0009195769493122056, 'learning_rate': 0.033995888275097745, 'subsample': 0.7141195140938823, 'colsample_bytree': 0.6582740117481474, 'max_depth': 3, 'min_child_weight': 2, 'eta': 0.0002139882676774764, 'gamma': 7.545734627111594e-07, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 206.55344990167336.
[I 2024-06-20 04:38:27,828] Trial 1 finished with value: 197.72858231992535 and parameters: {'booster': 'gbtree', 'lambda': 0.020058164042914165, 'alpha': 0.0003068265068140188, 'learning_rate': 0.023923058442758726, 'subsample': 0.43667065093233326, 'colsample_bytree': 0.9905460899757074, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.38634607785108876, 'gamma': 2.0537233540210722e-08, 'grow_policy': 'lossguide

Trial: 0 - MSE: 206.55344990167336 - RMSE: 14.371967502804665
Trial: 1 - MSE: 197.72858231992535 - RMSE: 14.061599564769484


[I 2024-06-20 04:38:27,871] Trial 2 finished with value: 174.6490765665892 and parameters: {'booster': 'gblinear', 'lambda': 7.351341839386231e-08, 'alpha': 1.189322314639463e-06, 'learning_rate': 0.01007761970818487, 'subsample': 0.8188231148048841, 'colsample_bytree': 0.5931626832684884}. Best is trial 2 with value: 174.6490765665892.
[I 2024-06-20 04:38:27,888] Trial 3 finished with value: 167.28159273003376 and parameters: {'booster': 'gblinear', 'lambda': 5.0136508997625806e-08, 'alpha': 9.177983810967162e-07, 'learning_rate': 0.0050026244791997925, 'subsample': 0.5399525258898941, 'colsample_bytree': 0.9921812861639546}. Best is trial 3 with value: 167.28159273003376.
[I 2024-06-20 04:38:27,962] Trial 4 finished with value: 172.50000328192306 and parameters: {'booster': 'gbtree', 'lambda': 3.313561587452981e-08, 'alpha': 7.649920304141375e-08, 'learning_rate': 0.00802174786969275, 'subsample': 0.6041878529955051, 'colsample_bytree': 0.6966931198521289, 'max_depth': 7, 'min_child_

Trial: 2 - MSE: 174.6490765665892 - RMSE: 13.215486240263322
Trial: 3 - MSE: 167.28159273003376 - RMSE: 12.933738544211947
Trial: 4 - MSE: 172.50000328192306 - RMSE: 13.133925661504373
Trial: 5 - MSE: 206.1351545997448 - RMSE: 14.357407655971352
Trial: 6 - MSE: 166.73012039687853 - RMSE: 12.912401805894925
Trial: 7 - MSE: 195.87987219455144 - RMSE: 13.99570906365774
Trial: 8 - MSE: 163.3586853003394 - RMSE: 12.781184815983979
Trial: 9 - MSE: 166.1514505646553 - RMSE: 12.88997480853455


[I 2024-06-20 04:38:28,125] Trial 10 finished with value: 161.34359121784934 and parameters: {'booster': 'dart', 'lambda': 0.874777346293958, 'alpha': 1.0166774215650573e-08, 'learning_rate': 0.0010541858874568862, 'subsample': 0.21367381413560954, 'colsample_bytree': 0.39764645433663504, 'max_depth': 7, 'min_child_weight': 6, 'eta': 3.123572781379101e-05, 'gamma': 2.0897512660424668e-06, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.7682776692921705, 'skip_drop': 4.222922474113876e-08}. Best is trial 10 with value: 161.34359121784934.
[I 2024-06-20 04:38:28,189] Trial 11 finished with value: 161.74222129273267 and parameters: {'booster': 'dart', 'lambda': 0.8391492884184247, 'alpha': 1.3239228231875898e-08, 'learning_rate': 0.001351842461098761, 'subsample': 0.20130932785790845, 'colsample_bytree': 0.38062149414561786, 'max_depth': 7, 'min_child_weight': 6, 'eta': 2.6471622273280614e-05, 'gamma': 1.9672517223879417e-06, 'grow_policy':

...

Trial: 92 - MSE: 160.3317613075732 - RMSE: 12.662217866849915
Trial: 93 - MSE: 160.33426659495174 - RMSE: 12.662316794131781
Trial: 94 - MSE: 160.25616897457965 - RMSE: 12.659232558673517
Trial: 95 - MSE: 160.32915462379862 - RMSE: 12.66211493486766


[I 2024-06-20 04:38:34,571] Trial 96 finished with value: 160.26255386749295 and parameters: {'booster': 'gbtree', 'lambda': 2.591438044554292e-08, 'alpha': 0.0013895216032974425, 'learning_rate': 0.001145069437319859, 'subsample': 0.563257634319806, 'colsample_bytree': 0.07298072147600378, 'max_depth': 7, 'min_child_weight': 2, 'eta': 1.7512863315467436e-07, 'gamma': 1.3690716887841807e-05, 'grow_policy': 'lossguide'}. Best is trial 94 with value: 160.25616897457965.
[I 2024-06-20 04:38:34,666] Trial 97 finished with value: 160.32304740755353 and parameters: {'booster': 'gbtree', 'lambda': 2.7573395248860465e-08, 'alpha': 0.001253426085787693, 'learning_rate': 0.0013632938941770987, 'subsample': 0.5569502982767511, 'colsample_bytree': 0.07412683266174995, 'max_depth': 7, 'min_child_weight': 2, 'eta': 4.434916702116717e-07, 'gamma': 1.2466362819231841e-05, 'grow_policy': 'lossguide'}. Best is trial 94 with value: 160.25616897457965.
[I 2024-06-20 04:38:34,725] Trial 98 finished with va

Trial: 96 - MSE: 160.26255386749295 - RMSE: 12.659484739415461
Trial: 97 - MSE: 160.32304740755353 - RMSE: 12.661873771585054
Trial: 98 - MSE: 160.53449248137917 - RMSE: 12.67022069584343


[I 2024-06-20 04:38:34,788] Trial 99 finished with value: 160.32377414561387 and parameters: {'booster': 'gbtree', 'lambda': 1.4764381624728027e-08, 'alpha': 0.006639386111854284, 'learning_rate': 0.0013709078573252993, 'subsample': 0.5274810369675217, 'colsample_bytree': 0.0718003803519881, 'max_depth': 7, 'min_child_weight': 2, 'eta': 2.356617559467337e-07, 'gamma': 2.719117999570968e-05, 'grow_policy': 'lossguide'}. Best is trial 94 with value: 160.25616897457965.


Trial: 99 - MSE: 160.32377414561387 - RMSE: 12.661902469440122


In [16]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida: {trial.value}')

Número de pruebas: 100
Mejor prueba: 94
Mejores parametros: {'booster': 'gbtree', 'lambda': 6.345455086977928e-08, 'alpha': 0.0014098433736486112, 'learning_rate': 0.0011259621834358992, 'subsample': 0.5661590070757975, 'colsample_bytree': 0.07316442112874563, 'max_depth': 7, 'min_child_weight': 2, 'eta': 1.6756116039536554e-07, 'gamma': 1.9909003535176968e-05, 'grow_policy': 'lossguide'}
Mejor valor de pérdida: 160.25616897457965


## Single Thread

In [17]:
# configuration optuna
study_st = optuna.create_study(direction='minimize')
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test), n_trials=n_trials)

[I 2024-06-20 04:38:34,834] A new study created in memory with name: no-name-3fbe27dd-d9d0-437b-9fcc-524a7fe40ea5
[I 2024-06-20 04:38:35,002] Trial 0 finished with value: 44.951809255377164 and parameters: {'booster': 'dart', 'lambda': 4.502531508562798e-05, 'alpha': 2.5433415230861575e-06, 'learning_rate': 0.008413846032761697, 'subsample': 0.3308606705503636, 'colsample_bytree': 0.1126548499165686, 'max_depth': 3, 'min_child_weight': 5, 'eta': 1.5628499411243148e-07, 'gamma': 0.001400345781001432, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 4.383249839009795e-05, 'skip_drop': 7.517358674552468e-06}. Best is trial 0 with value: 44.951809255377164.


Trial: 0 - MSE: 44.951809255377164 - RMSE: 6.704611044302061


[I 2024-06-20 04:38:35,125] Trial 1 finished with value: 45.963203640022776 and parameters: {'booster': 'dart', 'lambda': 0.000341125928998226, 'alpha': 1.3513438132270579e-06, 'learning_rate': 0.00551974161343263, 'subsample': 0.5964790271280369, 'colsample_bytree': 0.30513720730911753, 'max_depth': 3, 'min_child_weight': 9, 'eta': 0.00033005456782818517, 'gamma': 1.9401582708187025e-08, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 2.4212642855417116e-08, 'skip_drop': 0.013258027492347566}. Best is trial 0 with value: 44.951809255377164.


Trial: 1 - MSE: 45.963203640022776 - RMSE: 6.779616776781913


[I 2024-06-20 04:38:35,209] Trial 2 finished with value: 48.16002765307305 and parameters: {'booster': 'dart', 'lambda': 0.40827671843861824, 'alpha': 5.490643286815762e-08, 'learning_rate': 0.009457528435941519, 'subsample': 0.4277315413892853, 'colsample_bytree': 0.7408205134021052, 'max_depth': 3, 'min_child_weight': 7, 'eta': 0.0005913887777654375, 'gamma': 0.007558967960345543, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 6.728822582555352e-07, 'skip_drop': 1.5758861786534337e-07}. Best is trial 0 with value: 44.951809255377164.
[I 2024-06-20 04:38:35,249] Trial 3 finished with value: 61.68964778017931 and parameters: {'booster': 'dart', 'lambda': 1.305952558526151e-05, 'alpha': 0.0007210974461849, 'learning_rate': 0.0466771082536469, 'subsample': 0.9899992685035357, 'colsample_bytree': 0.21735174439294658, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.6578361226186004, 'gamma': 1.2053583547184215e-05, 'grow_policy': 'depthwise', 

...

Trial: 87 - MSE: 44.23986815152903 - RMSE: 6.651305747861019
Trial: 88 - MSE: 44.32928007372741 - RMSE: 6.658023736344548
Trial: 89 - MSE: 45.913738247059705 - RMSE: 6.775967698200731
Trial: 90 - MSE: 48.78399655497275 - RMSE: 6.984554141459049


[I 2024-06-20 04:38:40,315] Trial 91 finished with value: 43.85086913558739 and parameters: {'booster': 'gbtree', 'lambda': 2.788523027594717e-07, 'alpha': 0.003402788133101675, 'learning_rate': 0.0024812710765380825, 'subsample': 0.23076589699921232, 'colsample_bytree': 0.13260390074593476, 'max_depth': 3, 'min_child_weight': 8, 'eta': 3.781348889136817e-07, 'gamma': 0.1269891908969345, 'grow_policy': 'lossguide'}. Best is trial 58 with value: 43.77194039249671.
[I 2024-06-20 04:38:40,361] Trial 92 finished with value: 43.80473206045157 and parameters: {'booster': 'gbtree', 'lambda': 4.096588797751153e-07, 'alpha': 0.004148296119107991, 'learning_rate': 0.0020146433824108456, 'subsample': 0.224991012444325, 'colsample_bytree': 0.09027424276432112, 'max_depth': 3, 'min_child_weight': 8, 'eta': 2.60238580754273e-08, 'gamma': 0.08821621046993038, 'grow_policy': 'lossguide'}. Best is trial 58 with value: 43.77194039249671.
[I 2024-06-20 04:38:40,407] Trial 93 finished with value: 43.79697

Trial: 91 - MSE: 43.85086913558739 - RMSE: 6.621998877649209
Trial: 92 - MSE: 43.80473206045157 - RMSE: 6.618514339370398
Trial: 93 - MSE: 43.796978225470745 - RMSE: 6.617928544905176
Trial: 94 - MSE: 43.80395050779483 - RMSE: 6.618455296199773
Trial: 95 - MSE: 43.801184848175204 - RMSE: 6.618246357470777


[I 2024-06-20 04:38:40,543] Trial 96 finished with value: 43.79272359000065 and parameters: {'booster': 'gbtree', 'lambda': 5.958998560601568e-07, 'alpha': 0.01643326122103283, 'learning_rate': 0.0019247638854574497, 'subsample': 0.29635815778788305, 'colsample_bytree': 0.0882679803140892, 'max_depth': 3, 'min_child_weight': 9, 'eta': 3.9201546044949865e-08, 'gamma': 0.029286994086984222, 'grow_policy': 'lossguide'}. Best is trial 58 with value: 43.77194039249671.
[I 2024-06-20 04:38:40,589] Trial 97 finished with value: 43.79789326613098 and parameters: {'booster': 'gbtree', 'lambda': 7.529306818120728e-07, 'alpha': 0.015537577596299567, 'learning_rate': 0.001972453381095098, 'subsample': 0.29399735172696245, 'colsample_bytree': 0.08558671320851756, 'max_depth': 3, 'min_child_weight': 9, 'eta': 3.47768010495174e-08, 'gamma': 0.03559120758105356, 'grow_policy': 'lossguide'}. Best is trial 58 with value: 43.77194039249671.
[I 2024-06-20 04:38:40,635] Trial 98 finished with value: 43.754

Trial: 96 - MSE: 43.79272359000065 - RMSE: 6.617607089424443
Trial: 97 - MSE: 43.79789326613098 - RMSE: 6.617997678008884
Trial: 98 - MSE: 43.75490417724878 - RMSE: 6.614748988226899
Trial: 99 - MSE: 43.75809531945613 - RMSE: 6.6149901979863985


In [18]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 100
Mejor prueba: 98
Mejores parametros: {'booster': 'gbtree', 'lambda': 8.935136296496447e-07, 'alpha': 0.018029913163782663, 'learning_rate': 0.0015196561728450498, 'subsample': 0.2953169350070087, 'colsample_bytree': 0.08910672952971163, 'max_depth': 3, 'min_child_weight': 9, 'eta': 3.41411575494955e-08, 'gamma': 0.010141306114056094, 'grow_policy': 'lossguide'}
Mejor valor de pérdida en validación: 43.75490417724878


## Multi Thread

In [19]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize')
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test), n_trials=n_trials)

[I 2024-06-20 04:38:40,713] A new study created in memory with name: no-name-438555b0-184d-4ff3-8587-178e2bb35573
[I 2024-06-20 04:38:40,850] Trial 0 finished with value: 696.9365992300261 and parameters: {'booster': 'dart', 'lambda': 0.00011423713210046168, 'alpha': 0.0036051565253115837, 'learning_rate': 0.014767471632996495, 'subsample': 0.5184648608027747, 'colsample_bytree': 0.2913250871478764, 'max_depth': 7, 'min_child_weight': 10, 'eta': 0.0005027344154438699, 'gamma': 0.04239732193236443, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 2.3770540966430764e-07, 'skip_drop': 1.816997801247694e-05}. Best is trial 0 with value: 696.9365992300261.


Trial: 0 - MSE: 696.9365992300261 - RMSE: 26.399556799878784


[I 2024-06-20 04:38:40,999] Trial 1 finished with value: 653.6068980765482 and parameters: {'booster': 'gbtree', 'lambda': 0.19429245885968427, 'alpha': 0.26061692676197384, 'learning_rate': 0.0033226809941971696, 'subsample': 0.3532160404420888, 'colsample_bytree': 0.054048752154101015, 'max_depth': 9, 'min_child_weight': 8, 'eta': 1.9121214005400837e-07, 'gamma': 1.4516489352027327e-08, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 653.6068980765482.
[I 2024-06-20 04:38:41,050] Trial 2 finished with value: 654.3147723311791 and parameters: {'booster': 'dart', 'lambda': 0.01687659825485451, 'alpha': 8.019462154550141e-08, 'learning_rate': 0.00103688064545031, 'subsample': 0.39836986039570077, 'colsample_bytree': 0.6734848770013253, 'max_depth': 7, 'min_child_weight': 6, 'eta': 0.2726506934787567, 'gamma': 0.666830000967471, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 3.9104188620445127e-05, 'skip_drop': 3.459152868978455e-05

Trial: 1 - MSE: 653.6068980765482 - RMSE: 25.565736799015752
Trial: 2 - MSE: 654.3147723311791 - RMSE: 25.57957725082999


[I 2024-06-20 04:38:41,095] Trial 3 finished with value: 786.4524853853994 and parameters: {'booster': 'dart', 'lambda': 0.09011152163281694, 'alpha': 1.3467905603041055e-08, 'learning_rate': 0.03982564078513162, 'subsample': 0.5269034522123204, 'colsample_bytree': 0.9935451330335564, 'max_depth': 7, 'min_child_weight': 4, 'eta': 4.46216234866124e-06, 'gamma': 0.00013519999797410496, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 1.1163124503347211e-07, 'skip_drop': 0.47523096306014617}. Best is trial 1 with value: 653.6068980765482.
[I 2024-06-20 04:38:41,101] Trial 4 finished with value: 651.2827633774873 and parameters: {'booster': 'gblinear', 'lambda': 0.0616851618077235, 'alpha': 0.003567365441465661, 'learning_rate': 0.0012416452353680018, 'subsample': 0.44303892497347813, 'colsample_bytree': 0.45562423390234524}. Best is trial 4 with value: 651.2827633774873.
[I 2024-06-20 04:38:41,135] Trial 5 finished with value: 655.88620928840

...

Trial: 86 - MSE: 777.4779205256672 - RMSE: 27.88329106338897
Trial: 87 - MSE: 651.9127128961683 - RMSE: 25.532581398992313
Trial: 88 - MSE: 676.9639537441151 - RMSE: 26.018530968217924
Trial: 89 - MSE: 650.0136810572371 - RMSE: 25.495365874159113


[I 2024-06-20 04:38:44,630] Trial 90 finished with value: 655.3293350293768 and parameters: {'booster': 'gblinear', 'lambda': 0.0441089388869547, 'alpha': 0.24190641140247535, 'learning_rate': 0.009599327150877681, 'subsample': 0.8462128427768273, 'colsample_bytree': 0.6987828160751359}. Best is trial 65 with value: 647.4025069005219.
[I 2024-06-20 04:38:44,735] Trial 91 finished with value: 651.0028478497898 and parameters: {'booster': 'gblinear', 'lambda': 1.2167389250739177e-05, 'alpha': 0.4174077087113444, 'learning_rate': 0.0036368705463924417, 'subsample': 0.8991932542603941, 'colsample_bytree': 0.543725489255162}. Best is trial 65 with value: 647.4025069005219.
[I 2024-06-20 04:38:44,760] Trial 92 finished with value: 648.4189079887958 and parameters: {'booster': 'gblinear', 'lambda': 7.0391331724536655e-06, 'alpha': 0.9927100710549435, 'learning_rate': 0.004176707772346873, 'subsample': 0.8649059222742658, 'colsample_bytree': 0.43838016975713084}. Best is trial 65 with value: 6

Trial: 90 - MSE: 655.3293350293768 - RMSE: 25.59940106778627
Trial: 91 - MSE: 651.0028478497898 - RMSE: 25.51475745230179
Trial: 92 - MSE: 648.4189079887958 - RMSE: 25.46407092333816
Trial: 93 - MSE: 732.0791900752816 - RMSE: 27.056961952061094
Trial: 94 - MSE: 648.4586173481391 - RMSE: 25.464850624893504
Trial: 95 - MSE: 651.0376591761474 - RMSE: 25.51543962341522


[I 2024-06-20 04:38:44,835] Trial 95 finished with value: 651.0376591761474 and parameters: {'booster': 'gblinear', 'lambda': 3.0297699094208917e-06, 'alpha': 0.38525112349658547, 'learning_rate': 0.0032155620476452666, 'subsample': 0.8051808386785965, 'colsample_bytree': 0.4398550721269984}. Best is trial 65 with value: 647.4025069005219.
[I 2024-06-20 04:38:44,861] Trial 96 finished with value: 653.5874888804104 and parameters: {'booster': 'gblinear', 'lambda': 1.7981258694501888e-06, 'alpha': 0.12797290916345683, 'learning_rate': 0.005819618169641675, 'subsample': 0.9377183042331332, 'colsample_bytree': 0.40059813720190124}. Best is trial 65 with value: 647.4025069005219.
[I 2024-06-20 04:38:44,885] Trial 97 finished with value: 655.2108595215525 and parameters: {'booster': 'gblinear', 'lambda': 1.1410815213255624e-05, 'alpha': 4.6581660082447985e-06, 'learning_rate': 0.006664397712939782, 'subsample': 0.962925994472794, 'colsample_bytree': 0.3162121966828031}. Best is trial 65 with

Trial: 96 - MSE: 653.5874888804104 - RMSE: 25.565357202284705
Trial: 97 - MSE: 655.2108595215525 - RMSE: 25.597086934289074
Trial: 98 - MSE: 651.322431075337 - RMSE: 25.52101939726031
Trial: 99 - MSE: 651.9181188676823 - RMSE: 25.532687262951434


In [20]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 100
Mejor prueba: 65
Mejores parametros: {'booster': 'gblinear', 'lambda': 2.022940291246672e-07, 'alpha': 0.9984462311756452, 'learning_rate': 0.00805880113710561, 'subsample': 0.9756494846570188, 'colsample_bytree': 0.6217328071739047}
Mejor valor de pérdida en validación: 647.4025069005219


# Training

## General

In [21]:
# hyperparameters
study_g.best_trial.params

{'booster': 'gbtree',
 'lambda': 6.345455086977928e-08,
 'alpha': 0.0014098433736486112,
 'learning_rate': 0.0011259621834358992,
 'subsample': 0.5661590070757975,
 'colsample_bytree': 0.07316442112874563,
 'max_depth': 7,
 'min_child_weight': 2,
 'eta': 1.6756116039536554e-07,
 'gamma': 1.9909003535176968e-05,
 'grow_policy': 'lossguide'}

In [22]:
# general models
dtrain = xgb.DMatrix(X_g_train, label=y_g_train)
dtest = xgb.DMatrix(X_g_test, label=y_g_test)
# training
model = xgb.train(study_g.best_trial.params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_g_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)}")

MSE: 160.25693599458833 - RMSE: 12.65926285352304


## Single Thread

In [23]:
# hyperparameters
study_st.best_trial.params

{'booster': 'gbtree',
 'lambda': 8.935136296496447e-07,
 'alpha': 0.018029913163782663,
 'learning_rate': 0.0015196561728450498,
 'subsample': 0.2953169350070087,
 'colsample_bytree': 0.08910672952971163,
 'max_depth': 3,
 'min_child_weight': 9,
 'eta': 3.41411575494955e-08,
 'gamma': 0.010141306114056094,
 'grow_policy': 'lossguide'}

In [24]:
# single thread model
dtrain = xgb.DMatrix(X_st_train, label=y_st_train)
dtest = xgb.DMatrix(X_st_test, label=y_st_test)
# training
model = xgb.train(study_st.best_trial.params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_st_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)}")

MSE: 43.76117185037321 - RMSE: 6.615222736263172


## Multi Thread

In [25]:
# hyperparameters
study_mm.best_trial.params

{'booster': 'gblinear',
 'lambda': 2.022940291246672e-07,
 'alpha': 0.9984462311756452,
 'learning_rate': 0.00805880113710561,
 'subsample': 0.9756494846570188,
 'colsample_bytree': 0.6217328071739047}

In [26]:
# multi thread model
dtrain = xgb.DMatrix(X_mm_train, label=y_mm_train)
dtest = xgb.DMatrix(X_mm_test, label=y_mm_test)
# training
model = xgb.train(study_mm.best_trial.params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_mm_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)}")

MSE: 647.4701157874348 - RMSE: 25.445434085262423


# Conclusion
Queda trabajo que hacer en la red, además de conseguir más datos para un entrenamiento más robusto. Queda por ahora descartado el uso de solo un modelo para multi-threading y single-threading, ya que el modelo tiene más del triple de *loss*.